In [1]:
### This is code modified from Pavel's initial electrical test code.
##Author: Chris
## This is a playground for trying out random connections and useful ideas with electrical testing on Keithley DAQ6510

from cgi import test
from optparse import Values
from sqlite3 import Row
from time import time
import tkinter as tk
from turtle import width
import pyvisa
import time 
# import keyboard
import time
import cv2
from PIL import Image, ImageTk
from webbrowser import get
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import threading as th
import minimalmodbus
from skopt import Optimizer
from skopt.plots import plot_objective
from skopt.plots import plot_evaluations
# from simple_pid import PID
import os
rm = pyvisa.ResourceManager()
print(rm.list_resources()) 





# Initialize Keithley DAQ6510 Multimeter:
com_mm = rm.open_resource('USB0::0x05E6::0x6510::04433485::INSTR')
com_mm.clear()
com_mm.write("*IDN?\n")
print(com_mm.read())

com_mm.clear()
com_mm.close()
com_mm = rm.open_resource('USB0::0x05E6::0x6510::04433485::INSTR')

#Attempt writing SCPI function:
script_name = "destroy"
com_mm.write("loadscript "+ script_name)

#Attempt writing TSP functions and controlling Keithley
#the script below works, but cannot delete scripts from the multimeter...
com_mm.write("reset()")
com_mm.write("beeper.beep(1, 600)")

#script_name = "W_sweep"
#com_mm.write("script.delete("+script_name+")") #THIS DOESNT WORK
com_mm.write("script.delete(string.format('W_sweep'))")
com_mm.write("endscript")

C:\Users\Osuji\AppData\Local\Temp\ipykernel_10364\1091973403.py:5: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  from cgi import test


('USB0::0x05E6::0x6510::04433485::INSTR', 'ASRL1::INSTR', 'ASRL3::INSTR', 'ASRL4::INSTR', 'ASRL7::INSTR', 'ASRL21::INSTR')
KEITHLEY INSTRUMENTS,MODEL DAQ6510,04433485,1.0.04b



11

In [2]:
com_mm.write("destroy.run()")
script_name = "W_sweep"
com_mm.write("loadscript "+ script_name)
com_mm.write("function W_sweep()")
#Load initial parameters
com_mm.write("reset()")
com_mm.write("scanCount=10")
com_mm.write("sampleCount=8")
com_mm.write("defbuffer1.capacity = scanCount * sampleCount")
#Set up each channel measurement, we want to sweep voltage, measure current
#Get current density - linear sweep voltametry
com_mm.write("channel.setdmm('101', dmm.ATTR_MEAS_FUNCTION, dmm.FUNC_4W_RESISTANCE)")
com_mm.write("beeper.beep(1, 600)")
com_mm.write("channel.setdmm('102', dmm.ATTR_MEAS_FUNCTION, dmm.FUNC_4W_RESISTANCE)")
com_mm.write("beeper.beep(1, 800)")
com_mm.write("channel.setdmm('103', dmm.ATTR_MEAS_FUNCTION, dmm.FUNC_4W_RESISTANCE)")
com_mm.write("beeper.beep(1, 1000)")
com_mm.write("channel.setdmm('104', dmm.ATTR_MEAS_FUNCTION, dmm.FUNC_4W_RESISTANCE)")
com_mm.write("beeper.beep(1, 1200)")

com_mm.write('scan.add("101, 102, 103, 104")') #Load all eight channels
com_mm.write("scan.scancount = scanCount")
com_mm.write("print(string.format('Loaded scans'))")
com_mm.write("trigger.model.initiate()")
com_mm.write("print(string.format('Loaded scans'))")
com_mm.write("waitcomplete()")
##This is where we output results. Make choice on PRINT vs. save to disk
com_mm.write("i = 1")
com_mm.write("while i <= sampleCount do")
com_mm.write("delay(0.5)")
com_mm.write("myCnt = defbuffer1.n")
com_mm.write("printbuffer(i, myCnt, defbuffer1.readings)")
com_mm.write("i = myCnt + 1")
com_mm.write("end")
#Output results over. 
com_mm.write("end")
com_mm.write("endscript")


com_mm.write(script_name+".run()")
com_mm.write("W_sweep()")
#com_mm.write("destroy.run()")
time

<module 'time' (built-in)>

This next section is going to divvy up the functions that the AE setup can perform.
1. Function that measures eight electrical conductivity values at the same time. (Bulk measurements)
2. Function that uses the SoftAE code to make a decision based on...
    a. Drop shape?
    b. Conductivity? - This might be it. 
Things that are needed.
1. Positioning of PCB sections. 
2. Ordering of PCB sections - need to be consistent about indexing.
3. The ability to take a measurement value and store it in python, not just printing as code before has shown.
4. Co-opting AE code to fit a different measurement criterion.
5. Error returns/early ends for problems.
6. Making everything generic enough that it is distributable.
- Chris 09/24/2024

In [2]:
def multi_measure(num_measurements):
    """Function takes number of AE measurements, measures set of results.
    1. Initialize DAQ code.
    2. Measure on 8 channels.
    3. Output each channel into CSV"""
    #Section 1, write the DAQ communication code
    ##Note - we should decide if we initialize in here too. I say no and make a startup function.
    com_mm.write("destroy.run()")
    script_name = "W_sweep"
    com_mm.write("loadscript "+ script_name)
    com_mm.write("function W_sweep()")
    #Load initial parameters
    com_mm.write("reset()")
    com_mm.write("scanCount=10")
    com_mm.write("sampleCount=8")
    com_mm.write("defbuffer1.capacity = scanCount * sampleCount")
    #Set up each channel measurement, we want to sweep voltage, measure current
    #Get current density - linear sweep voltametry
    channel_list = []
    for i in range(num_measurements):
        channel = str(100+i+1) #We add the number of measurements to the index. Go from 101-108
        com_mm.write("channel.setdmm('"+channel+"', dmm.ATTR_MEAS_FUNCTION, dmm.FUNC_4W_RESISTANCE)")
        com_mm.write("beeper.beep(1, "+str(600+i*10)+")") 
        com_mm.write('scan.add("'+channel+'")') 


    #Load all eight channels
    com_mm.write("scan.scancount = scanCount")
    com_mm.write("print(string.format('Loaded scans'))")
    com_mm.write("trigger.model.initiate()")
    com_mm.write("print(string.format('Loaded scans'))") #WHERE DOES THIS GO? THE FUCNTIONS seem to work, but nothing is appearing on here or TSB
    com_mm.write("waitcomplete()")
    ##This is where we output results. Make choice on PRINT vs. save to disk
    #Output results over. 
    com_mm.write("end")
    com_mm.write("endscript")


    com_mm.write(script_name+".run()")
    com_mm.write("W_sweep()")
    #com_mm.write("destroy.run()")
    com_mm.write(("printbuffer(1, defbuffer1.n, defbuffer1.readings, defbuffer1.channels)"))
    raw_data = com_mm.read()
    print(raw_data)
    return


def measure_all():
    """This function just uses all channels on a multimeasure function"""
    """So you don't need to specify 8 every time."""
    multi_measure(8)
    return


def measure_specific(index):
    """Intermediate function for multi_measure_seq. if you want to measure things in order."""
    """Function takes number of AE measurements, measures set of results.
    1. Initialize DAQ code.
    2. Measure on a specific channel.
    3. Output each channel into CSV"""
    #Section 1, write the DAQ communication code
    ##Note - we should decide if we initialize in here too. I say no and make a startup function.
    com_mm.write("destroy.run()")
    script_name = "W_sweep"
    com_mm.write("loadscript "+ script_name)
    com_mm.write("function W_sweep()")
    PCB_2024 = {1:"101", 2:"102", 3:"103",4:"104",5:"105", 6:"106", 7:"107", 8:"108"} #I'm realizing that I did 1-8. We either want 0-7 or 1-8. This lines up nicely, even if it's not python-kosher
    #Load initial parameters
    com_mm.write("reset()")
    com_mm.write("scanCount=10")
    com_mm.write("sampleCount=1")
    com_mm.write("defbuffer1.capacity = scanCount * sampleCount")
    #Set up each channel measurement, we want to sweep voltage, measure current
    #Get current density - linear sweep voltametry
    channel = PCB_2024[index]
    com_mm.write("channel.setdmm('"+channel+"', dmm.ATTR_MEAS_FUNCTION, dmm.FUNC_4W_RESISTANCE)")
    com_mm.write("beeper.beep(1, "+str(600+int(channel))+")") 
    com_mm.write('scan.add("'+channel+'")') 


    #Load all eight channels
    com_mm.write("scan.scancount = scanCount")
    com_mm.write("print(string.format('Loaded scans'))")
    com_mm.write("trigger.model.initiate()")
    com_mm.write("print(string.format('Loaded scans'))") #WHERE DOES THIS GO? THE FUCNTIONS seem to work, but nothing is appearing on here or TSB
    com_mm.write("waitcomplete()")
    ##This is where we output results. Make choice on PRINT vs. save to disk
    #Output results over. 
    com_mm.write("end")
    com_mm.write("endscript")


    com_mm.write(script_name+".run()")
    com_mm.write("W_sweep()")
    #com_mm.write("destroy.run()")
    com_mm.write(("printbuffer(1, defbuffer1.n, defbuffer1.readings, defbuffer1.channels)"))
    #raw_data = com_mm.read()
    #print(raw_data)

def multi_measure_seq(num_measurements, var1, var2):
    """Function works with syringe pumps to apply a number of set parameters to x # of measurements - note, need a confirmation of all variables in syringe code.
    No AE here, just bulk measurements based on a set of supplied variables.
    1. Initialize drop locations.
    2. Initialize known drop parameters.
    3. In for loop for number of known drops:
    a. Move to location i
    b. Drop sample i
    c. Let dry.
    d. Measure electrical conductivity on correct channel.
    4. Do a final test of meauring all conductivities at end of measurement. Good check for any thixotropy."""

    # Initialize drop locations




    # Initialize drop parameters



    #For loop - drop, measure specific index, move on.

    for i in range(num_measurements-1): #Run up to 8 measurements
        print("Beginning run..."+str(i+1))
        #input SOFTAEDEMO movement
        #input softaedemo dropping
        #input softae picture
        measure_specific(i+1)
        print("Measurement done. Flushing...")
        #input softae flush
        #input softae stage movement
    #Measure all at once to check thixotropy
    measure_all()

    #Output 1-8 arrays for sequence of parameters. [index,resistivity,channel], then one total final measurement
    return



def multi_measure_AE(num_measurements, var1, var2, var3, var4):
    """Function that works with the drop-casting AE code to perform the first 8 measurements and store all information. Should be able to pick up where you left off
    after 8 measurements are completed. 
    General steps:
    1. move stage to correct location
    2. call either (a) initial dropcast parameters and move to 4 or (b) move to 3 if initial drop has already happened
    3. run AE optimization to determine parameters
    4. drop cast
    5. move and then clean/empty for next sample
    6. repeat"""
    #if num_measurements > 8:
        
    #Initialize stage



    #Initialize camera



    #Initialize dropcasting


    for i in range(num_measurements-1): #Run up to 8 measurements
        print("Beginning run..."+str(i+1))
        #input SOFTAEDEMO movement
        #input softaedemo dropping
        #input softae picture
        measure_specific(i+1)
        print("Measurement done. Flushing...")
        #input softae flush
        #input softae stage movement
        #Infomration such that the next run can change variables

    return

measure_all()
    

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

This is where multiple functions will be written that concern generic movement and intiialization. Make sure that these eventually make their way to a separate file
in the directory. 

First section is movement functions.

In [ ]:
def reset_loc():
    """This function resets to initial stage configuration."""
    
    #
    


def move_loc_PCB8_v1():
    """This function moves to a specific index on the PCB board with 8 channels
    The board used here is the location as of Oct. 2024. Create a new version of this function for every different stage."""

    #



def move_loc_DC_array_v1()
    """This function will """


The next section concerns data export code.

In [ ]:
def elec_export_csv():
    """This function takes an output from an electrical test and exports as a csv"""


def ____():

This third section contains functions for _____.


At the moment, it is currently just bundles of code to achieve a specific outcome.

In [25]:
#This code works if a buffer is filled with all 8 indices.
com_mm.write("scanCount=10")
com_mm.write("sampleCount=8")
com_mm.write("jeffrey= scanCount * sampleCount")
com_mm.write("printbuffer(1, defbuffer1.n, defbuffer1.readings, defbuffer1.channels)")
whoknows = com_mm.read()
real_data = whoknows.split(',')
